# Emerging Technologies

This notebook contains solutions for the assessment problems on classical and quantum algorithms using the Deutsch–Jozsa algorithm.


## Problem 1: Generating Random Boolean Functions

**Instructions:**  
Write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of constant or balanced functions taking four Boolean inputs.

In [10]:
# Problem 1 solution goes here

## Problem 2: Classical Testing for Function Type

**Instructions:**  
Write a Python function `determine_constant_balanced(f)` that analyzes a function f and returns "constant" or "balanced". Include a brief note on efficiency.

In [11]:
# Problem 2 solution goes here

## Problem 3: Quantum Oracles

**Instructions:**  
Using Qiskit, create quantum oracles for each of the possible single-Boolean-input functions in Deutsch's algorithm. Explain how each oracle works.

In [12]:
# Problem 3 solution goes here

## Problem 4: Deutsch's Algorithm with Qiskit

**Instructions:**  
Design a quantum circuit that solves Deutsch's problem for single-input functions. Demonstrate its use with each oracle and explain the interference pattern.

In [13]:
# Problem 4 solution goes here

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm

**Instructions:**  
Use Qiskit to create a quantum circuit for four-bit functions generated in Problem 1. Demonstrate it on constant and balanced functions and explain the results.

In [14]:
# Problem 5 solution goes here